# Low-salt-rejection Reverse Osmosis (LSRRO) Flowsheet
This tutorial shows how to build, initialize, simulate, and optimize an LSRRO flowsheet using WaterTAP.  

<center><img src="graphics/lsrro_flowsheet.png" width="800" height="400"></center>

In [ ]:
from watertap.flowsheets.lsrro.lsrro import (
    ACase,
    BCase,
    ABTradeoff,
    run_lsrro_case,
)
import logging

logging.getLogger("idaes").setLevel(logging.ERROR)
logging.getLogger("pyomo").setLevel(logging.ERROR)

## High-level Preview: Run an LSRRO optimization case. We will specify the following:
* 3-stage system (`number_of_stages` = 3)
* System Water Recovery of 50% (`water_recovery` = 0.50)
* 70 g/L TDS feed (`Cin` = 70) 
* Feed flow rate of 0.001 m3/s (`Qin` = 1e-3)


In [ ]:
m, results = run_lsrro_case(
    number_of_stages=3,  # total number of stages
    water_recovery=0.50,  # overall water recovery
    Cin=70,  # inlet NaCl conc in kg/m3,
    Qin=1e-3,  # inlet feed flowrate in m3/s
    permeate_quality_limit=1000e-6,  # permeate quality limit in ppm
    quick_start=True,  # skip rigorous initialization procedure for quick results (increased risk of non-convergence)
)

## Overview of setting up and optimizing the LSRRO flowsheet
* Step 0: Import Python libraries.

* Step 1: Build the LSRRO flowsheet.

* Step 2: Set specifications for LSRRO flowsheet.

* Step 3: Initialize the LSRRO flowsheet.

* Step 4: Optimize the LSRRO flowsheet.

* Step 5: Visualize results.

## Step 0: Import libraries.

In [ ]:
# Import LSRRO flowsheet module
from watertap.flowsheets.lsrro import lsrro

## Step 1: Build the LSRRO flowsheet.

In [ ]:
# Create a Pyomo concrete model, flowsheet, and NaCl property parameter block.
m = lsrro.build(
    number_of_stages=3,
    has_NaCl_solubility_limit=True,
    has_calculated_concentration_polarization=True,
    has_calculated_ro_pressure_drop=True,
    number_of_RO_finite_elements=10,
)

## Step 2: Set specifications for the LSRRO flowsheet.

In [ ]:
lsrro.set_operating_conditions(m)

In [ ]:
# Uncomment the following line to display the specified conditions of the model
# lsrro.display_operating_conditions(m)

## Step 3: Initialize the LSRRO flowsheet model.

In [ ]:
lsrro.initialize(m)

## Step 4: Simulate the LSRRO flowsheet.

In [ ]:
lsrro.solve(m, tee=False, raise_on_failure=True)

print("Simulation Results:\n")
lsrro.display_system(m)

In [ ]:
# store LCOW breakdown values from simulation results
lcow_breakdown_results, case_list = lsrro.get_lcow_breakdown(
    m, new_case="Simulated Results"
)

In [ ]:
lcow_breakdown_results

## Step 5: Optimize the LSRRO flowsheet.

In [ ]:
lsrro.optimize_set_up(
    m,
    water_recovery=0.5,
    A_case=ACase.optimize,  # water permeability coefficient
    B_case=BCase.optimize,  # salt permeability coefficient
    AB_tradeoff=ABTradeoff.equality_constraint,  # water and salt permeability equality constraints
    permeate_quality_limit=1000e-6,
)
optimization_results = lsrro.solve(m, raise_on_failure=True)

In [ ]:
print("Optimization Results:\n")
lsrro.display_system(m)

In [ ]:
lcow_breakdown_results, case_list = lsrro.get_lcow_breakdown(
    m,
    new_case="Optimization Results",
    existing_case_list=case_list,
    results=lcow_breakdown_results,
)

## Step 6: Visualize results

### Compare the breakdown of levelized cost of water for simulated and optimized results.


In [ ]:
lsrro.plot_lcow_breakdown(lcow_breakdown_results, scenario_labels=case_list)

## 

### Conduct sensitivity analysis on feed concentration and water recovery rate and visualize optimal levelized cost of water.
#### Running cases with feedwater concentration ranging from 70 to 150 g/L TDS and recovery rates from 20% to 50%.

<div class="alert alert-block alert-info">
<b>Note:</b> Generating each of the following figures may take several minutes.
</div>

#### Results for a 3-stage LSRRO system

Generate a cost-optimal feed concentration vs recovery profile for a given number of stages.  

The `feed_concentration_recovery_profile` function defined in `lsrro.py` uses the `parameter_sweep` with feed concentration and water recovery as sweep parameters.

In [ ]:
results3, _, fig3, ax3 = lsrro.feed_concentration_recovery_profile(
    m=m, number_of_stages=3, points_per_sweep=5
)

## 

In [ ]:
import pandas as pd

# Show all rows and columns for 3-stage sweep results
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

results3

## Try It Yourself
**Task**: Generate a feed concentration recovery profile and show the tabulated results for a 5-stage system.

<details>
  <summary>Click the arrow for a hint!</summary>
    
Use the code for the 3-stage system as a starting point.
</details>

In [ ]:
# Build the 5-stage LSRRO model
m = lsrro.build(
    number_of_stages=5,
    has_NaCl_solubility_limit=True,
    has_calculated_concentration_polarization=True,
    has_calculated_ro_pressure_drop=True,
    number_of_RO_finite_elements=10,
)

# Set operating conditions, initialize, and solve
lsrro.set_operating_conditions(m)
lsrro.initialize(m)
lsrro.solve(m, tee=False, raise_on_failure=True)

# Optimize the 5-stage model
lsrro.optimize_set_up(
    m,
    water_recovery=0.5,
    A_case=ACase.optimize,  # water permeability coefficient
    B_case=BCase.optimize,  # salt permeability coefficient
    AB_tradeoff=ABTradeoff.equality_constraint,  # water and salt permeability equality constraints
    permeate_quality_limit=1000e-6,
)
optimization_results = lsrro.solve(m, raise_on_failure=True)

print("5-Stage Optimization Results:\n")
lsrro.display_system(m)

# Generate feed concentration vs. recovery profile for 5-stage model
results5, _, fig5, ax5 = lsrro.feed_concentration_recovery_profile(
    m=m, number_of_stages=5, points_per_sweep=3
)

# Display the tabulated sweep results
results5

For more details on functions like `get_lcow_breakdown`, `plot_lcow_breakdown`, and `feed_concentration_recovery_profile`, please refer to the LSRRO flowsheet file in the WaterTAP repository: https://github.com/watertap-org/watertap/blob/main/watertap/flowsheets/lsrro/lsrro.py.